Event                           Effect
update of revision attribute    sync
save cmd                        compare revisions |> persist delta revisions, replace modelview w section, sync
add partner_ref with id         create partner_ref,partner_ref_revision with id, persist, replace modelview w section, sync
delete partner_ref              delete/invalidate in db, replace modelview w section, sync
add product_item                create Product_item, persist, replace modelview, synchronize with client
delete product_item             delete/invalidate in db, replace modelview, synchronize with client
                

In [ ]:
push!(LOAD_PATH, "src")
using BitemporalPostgres, BitemporalReactive, JSON, LifeInsuranceDataModel, SearchLight, Test, TimeZones, ToStruct
ENV["SEARCHLIGHT_USERNAME"] = "bitemporalpostgres"
ENV["SEARCHLIGHT_PASSWORD"] = "jw8s0F49KL"

LifeInsuranceDataModel.connect()

c = find(Contract)[1]
h = c.ref_history
w = Workflow(type_of_entity="Contract",
    ref_history=h,
    tsw_validfrom=ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"),
)
update_entity!(w)

committed = JSON.parse(JSON.json(csection(c.id.value, now(tz"UTC"), ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"))))
changed = JSON.parse(JSON.json(csection(c.id.value, now(tz"UTC"), ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"))))


In [ ]:

changed["revision"]["description"] = "CR first mutation by GUI model"
changed["partner_refs"][1]["rev"]["description"] = "CPR first mutation by GUI model"
changed["product_items"][1]["revision"]["description"] = "PIR first mutation by GUI model"
changed["product_items"][1]["tariff_items"][1]["tariff_ref"]["rev"]["deferment"] = 9
changed["product_items"][1]["tariff_items"][1]["tariff_ref"]["rev"]["description"] = "TIR first mutation by GUI model"
changed["product_items"][1]["tariff_items"][1]["partner_refs"][1]["rev"]["description"] = "bubu|"

deltas = BitemporalReactive.compareModelStateContract(committed, changed)
for delta in deltas
    println(delta)
    prev = delta[1]
    curr = delta[2]
    update_component!(prev, curr, w)
end

In [ ]:

pending = csection(c.id.value, now(tz"UTC"), ZonedDateTime(2022, 11, 01, 12, 0, 1, 1, tz"UTC"),1)

@test(pending.revision.description == "CR first mutation by GUI model")
@test(pending.partner_refs[1].rev.description == "CPR first mutation by GUI model")
@test(pending.product_items[1].revision.description == "PIR first mutation by GUI model")
@test(pending.product_items[1].tariff_items[1].tariff_ref.rev.deferment == 9)
@test(pending.product_items[1].tariff_items[1].tariff_ref.rev.description == "TIR first mutation by GUI model")
@test(pending.product_items[1].tariff_items[1].partner_refs[1].rev.description == "bubu|")